<a href="https://colab.research.google.com/github/PrelaunchZeus2/Data-319-Model-Data-Based-Methods/blob/main/Problem%20Set%201/PS1Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### DATA 319, Summer 2024
#### Problem Set 1, Question 6
##### Based on: Stanford CS246 class - Colab 1 assignment

## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [5]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c741439269a0a4500887fe1257a878b4cf62b944d6af7297d54fbf8e826635d2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Your goal is to determine which letter is at the beginning of the most words in this body of text. A natural approach would be to write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words ("pro" and "ject").

**Hint:**
1. split only on space (' ') but not hyphen/dash ('-') or other symbols.
2. you may find spark functions explode (https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.explode.html) and split (https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.functions.split.html) helpful, but you don't need to restrict to them as long as you can satisfy your goal.

In [20]:
from pyspark.sql import functions as F
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [40]:
# YOUR CODE HERE
txt = spark.read.text('/content/pg100.txt') #Read in text file
words = txt.select(F.explode(F.split(F.regexp_replace(F.col("value"), '[^a-zA-Z\s]', ''), " ")).alias("word")) #Divide into words ignoring any non letter character
word_counts = words.groupBy("word").count() #Group by word and count
word_counts.show()

+---------+-----+
|     word|count|
+---------+-----+
|   online|    4|
|     AWAY|    1|
|    those|  466|
|      art|  835|
|     some| 1096|
|    still|  488|
|       By|  815|
| painters|    1|
|     hope|  351|
|   travel|   32|
|    cures|    8|
|   ransom|   53|
|    spoil|   25|
|  tresses|    3|
|      few|   62|
|   Cannot|   44|
|forgetful|    5|
|    Drink|    8|
|   harder|   10|
| tripping|    6|
+---------+-----+
only showing top 20 rows



In [41]:
#
top_words = word_counts.sort(F.col("count").desc())
top_words.show()

+----+------+
|word| count|
+----+------+
|    |508156|
| the| 23437|
|   I| 20371|
| and| 18661|
|  to| 15898|
|  of| 15812|
|   a| 12721|
| you| 12089|
|  my| 10832|
|  in|  9919|
|  is|  8321|
|that|  8078|
| not|  8042|
|  me|  7744|
| And|  7455|
|with|  6831|
|  it|  6738|
|  be|  6396|
| his|  6320|
|your|  6017|
+----+------+
only showing top 20 rows



Once you obtained the desired results, **submit your code and answer to the question in the assignment through Canvas.**